In [50]:
# https://stackoverflow.com/questions/43599018/is-there-a-way-to-get-matplotlib-path-contains-points-to-be-inclusive-of-boundar
#I do quite like this command in Jupiter notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
#It makes things wider and not waste the space on your screen

In [1]:
import pandas as pd
from shapely.geometry import Polygon
import shapely
poly0 = Polygon([(0,0), (1,0), (1,1)])
poly1 = Polygon([(0,0), (1,0), (0,1)])
polys = pd.DataFrame({'poly': [poly0,poly1]})
polys

,poly
0,"POLYGON ((0 0, 1 0, 1 1, 0 0))"
1,"POLYGON ((0 0, 1 0, 0 1, 0 0))"


In [2]:
points = pd.DataFrame({'point': [shapely.geometry.Point(x)
    for x in [(.5,.3), (.7,.5), (-1.2,4.), (1.,0.0)]]})
points = points['point']
points

0    POINT (0.5 0.3)
1    POINT (0.7 0.5)
2     POINT (-1.2 4)
3        POINT (1 0)
Name: point, dtype: object

In [3]:
intersects = points[points.map(lambda p: poly0.intersects(p))]
intersects

0    POINT (0.5 0.3)
1    POINT (0.7 0.5)
3        POINT (1 0)
Name: point, dtype: object

In [4]:
isx = polys['poly'].map(lambda p: p.intersects(points[1]))
isx

0     True
1    False
Name: poly, dtype: bool

In [5]:
from matplotlib.path import Path
poly = Path([(0,0), (1,0), (1,1)])
points = [(.5,.3), (.5,.7), (-1.2,4.), (.99,0.01)]
poly.contains_points(points)

array([ True, False, False,  True])

In [6]:
import importlib
from uafgi.nsidc import nsidc0481
from uafgi import gdalutil
importlib.reload(nsidc0481)
importlib.reload(gdalutil)
gdf = nsidc0481.load_grids()
gdf.loc[0]

grid                                              E71.75N
poly    POLYGON ((489950 -1962150, 581050 -1962150, 58...
wkt     PROJCS["WGS 84 / NSIDC Sea Ice Polar Stereogra...
Name: 0, dtype: object

In [17]:
import pyproj
the_wkt = gdf.loc[0].wkt
wgs84 = pyproj.CRS.from_epsg("4326")  # lon/lat coordinates
local = pyproj.CRS.from_string(gdf.loc[0].wkt)
proj = pyproj.Transformer.from_crs(wgs84,local,always_xy=True)
proj.transform(-14.,79.)
the_wkt

'PROJCS["WGS 84 / NSIDC Sea Ice Polar Stereographic North",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",70],PARAMETER["central_meridian",-45],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",SOUTH],AXIS["Northing",SOUTH],AUTHORITY["EPSG","3413"]]'

In [104]:
seldf = pd.read_pickle('selections_full.df')
#seldf.loc = pd.Series(list(shapely.geometry.Point(xy) for xy in zip(seldf.lon,seldf.lat)))

In [155]:
poly0 = gdf.loc[6].poly
points = pd.Series([shapely.geometry.Point(x,y) for x,y in zip(*proj.transform(seldf.lon.tolist(), seldf.lat.tolist()))],name='terminus')
#points
intersects = points[points.map(lambda p: poly0.intersects(p))]

In [156]:
intersects

14     POINT (-391054.059355962 -1474088.671339191)
16    POINT (-374977.3009849711 -1495830.212015945)
Name: terminus, dtype: object

In [135]:
type(intersects)

pandas.core.series.Series

In [136]:
len(intersects)

2

In [157]:
col1 = pd.Series(index=intersects.index, data=['E22'] * len(intersects),name='grid')

#pd.merge(intersects,col1)

In [158]:
col1

14    E22
16    E22
Name: grid, dtype: object

In [161]:
df=pd.concat([intersects,col1],axis=1)
df

,terminus,grid
14,POINT (-391054.059355962 -1474088.671339191),E22
16,POINT (-374977.3009849711 -1495830.212015945),E22


In [164]:
    selections = seldf

    # Load the grids 
    gdf = nsidc0481.load_grids()
    grids_s = list()
    for index, row in gdf.iterrows():

        # Transform glacier terminus locations to this grid's projection
        local = pyproj.CRS.from_string(row['wkt'])
        proj = pyproj.Transformer.from_crs(wgs84,local,always_xy=True)
        projected_points = pd.Series(
            index=selections.index,
            data=[shapely.geometry.Point(x,y) for x,y in zip(
                *proj.transform(selections.lon.tolist(), selections.lat.tolist()))]
            )

        # Find intersections between terminus locations and this grid
        # NOTE: intersects includes selections.index
        intersects = points[projected_points.map(lambda p: row['poly'].intersects(p))]

        grids_s.append(pd.Series(index=intersects.index, data=[row['grid']] * len(intersects),name='grid'))

    grids = pd.concat(grids_s, axis=0)


/Users/eafischer2/spack/var/spack/environments/pismip6-catalina/.spack-env/view/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [169]:
grids = grids.sort_index()
grids

1     E71.75N
2     E67.55N
4     W70.90N
5     W70.90N
6     W70.90N
6     W70.55N
10    W77.55N
13    W73.75N
14    W76.10N
15    W76.35N
16    W75.85N
16    W76.10N
17    W72.90N
18    E63.35N
18    E63.00N
19    E65.10N
20    E66.60N
21    W64.25N
23    W62.10N
Name: grid, dtype: object

In [166]:
type(grids)

pandas.core.series.Series

In [171]:
grids.index.duplicated()
#grids.loc[grids.index.duplicated()]

array([False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False])

In [ ]:
from osgeo import ogr,osr
driver = ogr.GetDriverByName('ESRI Shapefile')
src_ds = driver.Open('troughs/shp/fjord_outlines.shp')
src_lyr = src_ds.GetLayer()   # Put layer number or name in her
src_srs = src_lyr.GetSpatialRef()
dst_srs = osr.SpatialReference()
dst_srs.ImportFromWkt(the_wkt)
transform = osr.CoordinateTransformation(src_srs, dst_srs)

fjords_s = list()
if True:
    while True:
        feat = src_lyr.GetNextFeature()
        if feat is None:
            break
        poly = feat.GetGeometryRef()
        poly.Transform(transform)
        print(poly)
        fjords_s.append(poly)
        #poly.transform(transform)
        #print(type(poly))
        #print(feat.GetGeometryRef())
        #print('--------')

fjords = pd.Series(name='fjords',data=fjords_s)
fjords

In [10]:
import shapefile
reader = shapefile.Reader('troughs/shp/terminus_locations.shp')
shape = reader.shape(0)
shape

In [11]:
shape.points


[[-54.36425146640052, 72.8503844114822]]

In [29]:
from uafgi import shputil
importlib.reload(shputil)
#for rec in shputil.read_shapefile('troughs/shp/terminus_locations.shp', the_wkt):
#    print(rec)
print(shputil.read_df('troughs/shp/terminus_locations.shp', the_wkt))

               popname        ID  \
0  Upernavik Isstrom S  GGN0452S   
1  Upernavik Isstrom C  GGN0452C   
2  Upernavik Isstrom N  GGN0452N   

                                          _shape  
0  POINT (-304464.7388332954 -1846270.687993343)  
1   POINT (-299794.8973538086 -1837190.44067212)  
2   POINT (-296162.7984253188 -1824348.37660353)  


In [49]:
import shapefile
# Polygon shapefile we are updating.
# We must include a file extension in
# this case because the file name
# has multiple dots and pyshp would get
# confused otherwise.
file_name = 'troughs/shp/todel/terminus_locations.shp'
# Create a shapefile reader
r = shapefile.Reader(file_name)
# Create a shapefile writer
# using the same shape type
# as our reader
with shapefile.Writer('troughs/shp/terminus_locations.shp', shapeType=r.shapeType) as w:
 # Copy over the existing dbf fields
 w.fields = list(r.fields)
    
 # Copy over the existing dbf records
 for shaperec in r.iterShapeRecords():
    print(shaperec.record)
    w.record(*shaperec.record)
    w.shape(shaperec.shape)

 w.record('Nordenskiold Gl. N','NORD001')
 w.point(-59.073,75.834)
 w.record('Nordenskiold Gl. 2','NORD002')
 w.point(-51.17,68.304)
 w.record('Nordenskiold Gl. 3','NORD003')
 w.point(-28.05,73.117)
 w.record('Deception O Gl. CN','FISCH001')
 w.point(-33.399,67.624)
 w.record('Koge Bugt Gl. C','FISCH002')
 w.point(-41.164,65.17)
 w.record('Ukaasorsuaq','FISCH003')
 w.point(-48.721,61.958)
    
 # Add a new polygon
 #w.poly(parts=[[[-104,24],[-104,25],[-103,25],[-103,24],[-104,24]]])
 # Add a new dbf record for our polygon making sure we include
 # all of the fields in the original file (r.fields)
# w.record("STANLEY","TD","091022/1500","27","21","48","ep")
 # Overwrite the old shapefile or change the name and make a copy 
 #w.save('x.shp')
#w.close()

Record #-1: ['Upernavik Isstrom S', 'GGN0452S']
Record #-1: ['Upernavik Isstrom C', 'GGN0452C']
Record #-1: ['Upernavik Isstrom N', 'GGN0452N']


In [47]:
shapely.geometry.Point(1,2).y


2.0

In [7]:
class MyClass:
    def __getitem__(self, *args):
        print(args)
        return None
    def __setitem__(self, *args):
        print('xx ', args)
myc = MyClass()
myc[3:4,5,6] = (4,3)

xx  ((slice(3, 4, None), 5, 6), (4, 3))
